##  import Beautifulsoup, request, selenium 사용

In [13]:
import requests
from bs4 import BeautifulSoup, element
import chardet
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
import urllib3

## 네이버 크롤링 위해 고정 header 사용

In [27]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
resultList = []

## Chrome 드라이버 생성 , headless 모드(웹브라우저 실행없이)로 실행, SSSL 인증서 오류 무시

In [15]:
chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument('--ignore-certificate-errors') 

#   크롤링 함수 정의 
### 뉴스 검색 사이트 URL 부분의 검색명에 {회사 이름}를 넣어줌  
### 뉴스 제목, 뉴스 날짜, 언론사, 뉴스 내용을 크롤링

In [38]:
def start_crawl(start_date, end_date,company):
    global news_date
    start_date_nospace = start_date.replace('.', '')
    end_date_nopspace = end_date.replace('.', '')
    current_page = 1
    searching = True

    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(5)

    search_urlback = f"{start_date}&de={end_date}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{start_date_nospace}to{end_date_nopspace},a:all&start=1"
    search_urlfront = f"https://search.naver.com/search.naver?where=news&query={company}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds="
    search_url = search_urlfront + search_urlback
    print(search_url)
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 새로운 컨텐츠가 로드될 때까지 대기
    while True:
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            last_height = new_height
            continue 
        else:
            last_height = new_height
            break

    naver_html_src = driver.page_source
    naver_soup = BeautifulSoup(naver_html_src, 'html.parser')
    pages = naver_soup.find('div', {'class' : 'sc_page_inner'})
    news_list = naver_soup.select('li[class="bx"]')

    for news in news_list:
        time.sleep(2)
        
        # 뉴스 제목
        try :
            news_title = news.find('a', attrs={'class':'news_tit'}).get('title')
        except AttributeError:
            news_title = "No title available"
        news_url = news.find('a', attrs={'class':'news_tit'}).get('href')

        # 뉴스 날짜 
        find_date = news.find_all('span', attrs={'class':'info'})
        if len(find_date) > 1:
            # if find_date == "4주 전":
            #     news_date = 20240212
            # elif find_date == "3주 전":
            #     news_date = 20240219
            # elif find_date == "2주 전":
            #     news_date = 20240226
            # elif find_date == "1주 전":
            #     news_date = 20240303
            # elif find_date == "*일 전":
            #     news_date = 20240312
            # else :                
            news_date = find_date[1].text
        else: 
            news_date = find_date[0].text
            
        # 뉴스 발행사 
        try:
            news_publisher = news.find('a', attrs={'class':'info press'}).text
        except:
            try:
                news_publisher = news.find('a', attrs={'class':'info'}).text
            except:
                continue
            
        # 뉴스 내용
        news_summary = news.find('a', attrs={'class':'api_txt_lines dsc_txt_wrap'}).text
        news_summary = news_summary[0:50]
        
        news_text = []
        try:
            news_response = requests.get(news_url, headers=headers)
        except :
            continue

        if news_summary in str(news_response.text):
            news_soup = BeautifulSoup(news_response.text, 'html.parser')
        else:
            news_soup = BeautifulSoup(news_response.content.decode('euc-kr', 'replace'), 'html.parser')
            
        news_content = str(news_soup.text).split('\n')
        news_index = [i for i, s in enumerate(news_content) if news_summary in s]
        
        if news_index:
            news_index = news_index[0]
            punctuations = ['.', ',', '?', '!']
            count = 0
            if news_index != 0:
                for i in reversed(range(0, news_index)):
                    if any(text in news_content[i] for text in punctuations): 
                        news_text.append(news_content[i])
                        count = 0
                        #print("i", i, "count", count)
                        #print(news_content[i])
                    else:
                        count +=1
                    if count > 3:
                        break

            # print(news_index)

            if news_text:
                news_text.reverse()
            count = 0
            news_text.append(news_content[news_index])
            for i in range(news_index, len(news_content)):
                
                if any(text in news_content[i] for text in punctuations): 
                    news_text.append(news_content[i])
                    count = 0

                else:
                    count +=1
                if count > 3:
                    break
            
            news_text = ' '.join(news_text)
        resultList.append([news_date, news_title, news_text, news_publisher])
        df = pd.DataFrame(resultList)

        # updated_df = pd.concat([existing_df, df], ignore_index=True)
        df.to_csv(f'news_{company}1.csv', encoding='utf-8-sig', index=False)

    driver.quit()

    return news_text, news_date

# 함수 실행
### 시작날짜, 끝날짜, 회사이름
### 중간에 언론사 or 네이버 서버에 의해 네트워크 중지되었다는 오류 발생할 수도 있음-=> .py로 실행
### 

In [39]:
# 실행
start_crawl('20240101','20240102','동진쎄미켐')

https://search.naver.com/search.naver?where=news&query=동진쎄미켐&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=20240101&de=20240102&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20240101to20240102,a:all&start=1


Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq\\backend\\cython\\checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

# 크롤링 파일 합치기
# 파일들의 리스트를 얻음
# 각 파일을 DataFrame으로 읽은 후 리스트에 저장
# 병합된 DataFrame을 CSV 파일로 저장


In [40]:
import pandas as pd
import glob


file_list = glob.glob(f'news_*.csv')

dfs = [pd.read_csv(file) for file in file_list]

merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv(f'newslast.csv', encoding='utf-8-sig', index=False)

ValueError: No objects to concatenate

# installation

## konlpy를 위한 jdk 설치

https://www.oracle.com/kr/java/technologies/javase/javase8u211-later-archive-downloads.html

윈도우 > 시스템 환경 변수 편집 > 환경변수 > 시스템 변수 > 새로 만들기

변수 : JAVA_HOME         
값(jdk 설치 경로) : C:\Program Files\Java\jdk-1.8

윈도우 > 시스템 환경 변수 편집 > 환경변수 > 시스템 변수 > Path > 편집 > 새로 만들기             
                          
%JAVA_HOME%\bin;

윈도우 cmd에서 $java -version      
으로 설치 확인

In [44]:
## 라이브러리 설치

In [45]:
!pip install wordcloud
!pip install kss
!pip install konlpy
!pip install scikit-learn
!pip install selenium

# import

In [ ]:
import re
import os
import base64
import math
import numpy as np
from PIL import Image
from collections import Counter
from wordcloud import WordCloud, ImageColorGenerator
import certifi
import matplotlib.pyplot as plt
from matplotlib import font_manager
font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams['font.family'] = font_family
import warnings
warnings.filterwarnings(action='ignore')
import kss
from konlpy.tag import Okt, Kkma
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize 

# 필요한 파일 준비

## stopwords-ko.txt 파일과 같은 경로에 다운로드

## 파일과 같은 경로에 wordcloud_mask.png 이름의 wordcloud 생성을 위한 mask 이미지 저장

# Text에서 stopwords 제거 및 문장 자르기

In [ ]:
class TextAnalyzer:
    def __init__(self):
        self.okt = Okt()
        self.kkma = Kkma()
        self.stopwords = []
        stopwords_path = './stopwords-ko.txt'
        with open(stopwords_path, 'r', encoding='UTF8') as file:
            for line in file:
                self.stopwords.append(line.strip())

    def text2sentences(self, text):
        sentences = kss.split_sentences(text)
        return sentences
        
    def sentences2nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            for word in self.okt.nouns(str(sentence)):
                if word in self.stopwords or len(word) <= 1:
                    continue    
                if word in nouns:
                    continue
                nouns.append(word)
        return nouns

#  단어 분석

In [ ]:
class WordAnalyzer:
    def __init__(self):
        self.text_analyze = TextAnalyzer()
        self.countvec = CountVectorizer()
        self.okt = Okt()
        self.stoptags = ['Determiner', 'Adverb', 'Conjunction', 'Exclamation', 'Josa']
        self.counttags = ['Noun', 'Verb', 'Adjective']
        self.essential_josa = ['은', '는', '이', '가', '을', '를']
        
    def build_word_graph(self, sentence):
        countvec_mat = normalize(self.countvec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.countvec.vocabulary_
        return np.dot(countvec_mat.T, countvec_mat), {vocab[word]: word for word in vocab}
    
    def get_ranks(self, graph, d=0.85):
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0    
            link_sum = np.sum(A[:, id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B)
        return {idx: r[0] for idx, r in enumerate(ranks)}

    # 키워드
    def text2keywords(self, text, word_num=15):        
        sentences = self.text_analyze.text2sentences(text)
        nouns = self.text_analyze.sentences2nouns(sentences)
        word_graph, idx2word = self.build_word_graph(nouns)
        word_rank_idx = self.get_ranks(word_graph)
        sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
        
        keywords = []
        index = []
        
        for idx in sorted_word_rank_idx[:word_num]:
            index.append(idx)
        for idx in index:
            keywords.append(idx2word[idx])
        return keywords
    
    # 품사 태깅
    def text2postag(self, text):
        postag = self.okt.pos(text)
        return postag
    
    # 빈도수 높은 단어
    def text2countwords(self, text):
        postag = self.text2postag(text)
        countwords_postag = []
        
        for i in range(len(postag)):
            if postag[i][1] in self.counttags:
                countwords_postag.append(postag[i][0])
        return countwords_postag
    
    # visualization
    def words2wordscount(self, words, counttype):
        if counttype == 'individual':
            whole = []
            for i in range(len(words)):
                for j in range(0, len(words)-i, 1):
                    whole.append(words[i])
            words = whole
        else:
            pass

        # 모든 경우에서 count를 통해 dict 생성
        count = Counter(words)
        wordscount = dict(count.most_common())
        # countwoords 선정시 단어 길이, 빈도수 1인 단어 제거
        if counttype == 'count':
            dict_key = list(wordscount.keys())
            for i in dict_key:
                if wordscount[i] == 1:
                    del(wordscount[i])
        else:
            pass
        return wordscount
    


    # 워드클라우드 시각화
    def visualize_wordcloud(self, text, wordtype):
        #wordcloud_path = os.getcwd()+'\\server\\routers\\text_analysis'
        wordcloud_path = '.'
        img_save_path = '.'
        if wordtype == 'keywords':
            mask = np.array(Image.open(wordcloud_path+'/기영이.jpg'))
            words = self.text2keywords(text)
            words = self.words2wordscount(words, 'individual')
        
        elif wordtype == 'countwords':
            mask = np.array(Image.open(wordcloud_path+'/기영이.jpg'))
            words = self.text2countwords(text)
            words = self.words2wordscount(words, 'count')
        
        image_colors = ImageColorGenerator(mask)
        wordcloud = WordCloud(font_path=font_fname, background_color='white',
                              mask=mask, width=mask.shape[1], height=mask.shape[0], prefer_horizontal=0.99999)
        cloud = wordcloud.generate_from_frequencies(words)
        
        plt.figure(figsize=(8,8))
        plt.imshow(cloud.recolor(color_func=image_colors), interpolation='bilinear')
        plt.axis('off')
        plt.savefig(img_save_path+f'/text_wordcloud_{wordtype}.png')


## 데이터 전처리

In [ ]:
import pandas as pd
company = "피에스케이"
news = pd.read_csv(f'news{company}last.csv', encoding='utf-8-sig')
news_title = news["1"].str.strip()
news_title_value = ' '.join(news_title.values)

# 실행

In [47]:
if __name__ == '__main__':
    stttext = news_title_value
    img_save_path = '-'
    
    TA = TextAnalyzer()     # 텍스트 분석 클래스
    WA = WordAnalyzer()     # 단어 분석 클래스

    keywords      = WA.text2keywords(stttext)                   
    top3_keywords = list(WA.words2wordscount(keywords, 'individual'))[:3]         # 키워드 상위 3개
    countwords      = WA.text2countwords(stttext)                 
    top3_countwords = list(WA.words2wordscount(countwords, 'countwords'))[:3]     # 빈도수 높은 단어 상위 3개
    WA.visualize_wordcloud(stttext, 'keywords')       # 키워드 워드클라우드
    WA.visualize_wordcloud(stttext, 'countwords')     # 빈도수 높은 단어 워드클라우드


NameError: name 'news_title_value' is not defined

# pytorch, tensorflow 설치하기 싫으니 colab으로 실행 해준다
## 허깅페이스 api 사용 => pipeline("task", "모델명")
## 예시 sentiment_classification("today is payday!")

In [ ]:
from transformers import pipeline

sentiment_classification = pipeline("text-classification", "snunlp/KR-FinBert-SC")

import pandas as pd 
# sentiment_classification("today is payday!")

## 해당하는 회사 이름 csv 불러오기 (경로 확인하기, 파일을 꼭 colab에 업로드해 줘야함!!), 열 이름 변경

In [48]:
import re
company = "DB하이텍"
news = pd.read_csv(f'/content/news{company}last.csv', encoding='utf-8-sig')
news.rename(columns={news.columns[0] : '날짜', 
                          news.columns[1] : '제목', 
                          news.columns[2] : '내용', 
                          news.columns[3] : '언론사'})

## 데이터 전처리

In [ ]:
# 데이터 전처리 함수 정의
def preprocess_text(text):
    # 괄호와 괄호 안 문자 제거
    text = re.sub(r'\([^)]*\)', ' ', text)
    # '...' 제거
    text = text.replace('...', ' ')
    return text
news_data = news["1"].apply(preprocess_text)
print(news_data)

## 뉴스 제목과 뉴스 내용 뽑아오기
## 뉴스 데이터 중에서 해당 회사 이름이 포함된 내용만 가져오기

In [50]:
# 뉴스 제목 열 가져오기
news_title = news["1"].str.strip()
# print(news_title)
news_data = news_title.str.contains(company)
if news_data True:

  print(news_title)
# news_data = news_title.str.contains(company)
# news_title = news.loc[:, "1"]
# print(news_data)

# news_content = news[["1","2"]]
# print(news_content)


# news_title_value = ' '.join(news_title.values)
# news_title_value = news_title.values.tolist()
# print(news_title_value)

SyntaxError: invalid syntax (3855880390.py, line 5)

In [52]:
# 실행

In [ ]:
sentiment_classification(news_title_value)